In [20]:
import torch
import numpy as np
from transformers import BertTokenizer, BertModel
from sklearn.metrics.pairwise import cosine_similarity
import random
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time


# Inicializa navegador
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.get("https://contexto.me/en/")

print("🔄 Carregando BERT...")
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertModel.from_pretrained("bert-base-uncased")
model.eval()

# Embeddings cache
embeddings = {}
memory = {}

🔄 Carregando BERT...


In [ ]:
# Palavras genéricas para abrir sentidos semânticos
temas_exploratorios = [
    "object", "place", "person", "animal", "emotion",
    "food", "vehicle", "technology", "family", "music",
    "clothing", "plant", "profession", "color"
]
# "tool"
temas_usados = set()

# Função para obter embedding
def get_embedding(word):
    if word in embeddings:
        return embeddings[word]

    with torch.no_grad():
        inputs = tokenizer(word, return_tensors="pt", truncation=True)
        inputs = {k: v for k, v in inputs.items()}
        outputs = model(**inputs)
        emb = outputs.last_hidden_state.mean(dim=1).squeeze().numpy()
        embeddings[word] = emb
        return emb

# Adiciona chute manual
def adicionar_chute(palavra, score):
    print(f"Chute registrado: '{palavra}' com score {score}")
    memory[palavra] = score
    get_embedding(palavra)

# Sugestão adaptativa
def sugerir_proximo():
    print("\n🎯 Gerando novas sugestões...")

    if not memory:
        sugestao = random.choice(temas_exploratorios)
        temas_usados.add(sugestao)
        print(f"🧭 Começando exploração: {sugestao}")
        return [sugestao]

    # Classificação refinada por score
    perfeitos = [(p, s) for p, s in memory.items() if s < 50]
    otimos    = [(p, s) for p, s in memory.items() if 50 <= s < 200]
    bons      = [(p, s) for p, s in memory.items() if 200 <= s < 300]
    oks       = [(p, s) for p, s in memory.items() if 300 <= s < 500]
    ruins     = [(p, s) for p, s in memory.items() if s >= 500]

    todos_ruins = len(ruins) == len(memory)

    if todos_ruins:
        restantes = [t for t in temas_exploratorios if t not in temas_usados]
        if restantes:
            sugestao = random.choice(restantes)
            temas_usados.add(sugestao)
            print(f"🔍 Todos os chutes estão ruins. Explorando novo tema: {sugestao}")
            return [sugestao]
        else:
            print("⚠ Todos os temas exploratórios já foram usados. Chutando palavra genérica aleatória.")
            return [random.choice([w for w in tokenizer.vocab.keys() if w.isalpha() and len(w) > 4 and w not in memory])]

    # Refinamento inteligente com prioridade por nível
    base = []

    if len(perfeitos) >= 1:
        base = sorted(perfeitos, key=lambda x: x[1])  # usa todos
        print(f"💥 Refinando a partir de perfeitos ({len(base)}): {[p for p, _ in base]}")
    elif len(otimos) >= 1:
        base = sorted(otimos, key=lambda x: x[1])
        print(f"🌟 Refinando a partir de ótimos ({len(base)}): {[p for p, _ in base]}")

    elif len(bons) >= 1:
        base = sorted(bons, key=lambda x: x[1])
        print(f"👌 Refinando a partir de bons ({len(base)}): {[p for p, _ in base]}")

    elif len(oks) >= 1:
        base = sorted(oks, key=lambda x: x[1])
        print(f"🙂 Refinando a partir de oks ({len(base)}): {[p for p, _ in base]}")
    else:
        base = sorted(memory.items(), key=lambda x: x[1])[:2]
        print(f"🤔 Refinando com os menos ruins: {[p for p, _ in base]}")

    # Cálculo do vetor médio com pesos
    vetores = np.array([get_embedding(p) for p, _ in base])
    pesos = np.array([1 / (s + 1) for _, s in base])
    pesos = pesos / pesos.sum()
    vetor_medio = np.average(vetores, axis=0, weights=pesos)

    # Seleção de candidatos
    candidatos = [w for w in tokenizer.vocab.keys() if w.isalpha() and len(w) > 4 and w not in memory]
    random.shuffle(candidatos)
    candidatos = candidatos[:1000]

    melhor_palavra = None
    melhor_sim = -1
    dic_sim = {}
    for w in candidatos:
        vec = get_embedding(w)
        sim = cosine_similarity([vetor_medio], [vec])[0][0]
        dic_sim[w] = sim

    # Ordena o dic_sim pelo valor de similaridade (do maior para o menor)
    dic_sim_ordenado = sorted(dic_sim.items(), key=lambda x: x[1], reverse=True)
    palavras = dic_sim_ordenado[:10]  # pega as 10 melhores palavras
    lista_palavras = [p[0] for p in palavras]
    print(f"✅ Próxima sugestão: {melhor_palavra} (sim={melhor_sim:.4f})")
    return lista_palavras

def enviar_chute_site(palavra):
    print(f"\n{palavra}")
    palavra = str(palavra)
    # Preenche o campo
    input_box = driver.find_element(By.CSS_SELECTOR, "input.word")
    input_box.clear()
    input_box.send_keys(palavra)
    input_box.send_keys(Keys.ENTER)

    # Aguarda resposta
    time.sleep(1.5)

    # Verifica se apareceu a mensagem de palavra repetida
    try:
        msg_box = driver.find_element(By.CSS_SELECTOR, ".message-text")
        if "already guessed" in msg_box.text:
            print(f"⚠️ Palavra já testada: {palavra}")
            return palavra, None  # Ignorar
    except:
        pass  # nenhuma mensagem = segue normal

    # Lê o chute mais recente
    try:
        resultado = driver.find_element(By.CSS_SELECTOR, ".row-wrapper.current .row")
        texto = resultado.text.strip()
        palavra_retornada, score = texto.split()
        return palavra_retornada.lower(), int(score)
    except Exception as e:
        print("⚠️ Erro ao tentar ler o resultado:", e)
        return palavra, None

def reenviar_chutes_para_interface():
    print("\n📤 Reenviando chutes manuais para a interface do jogo...")
    for palavra in list(memory.keys()):
        _, score_checado = enviar_chute_site(palavra)  # reenviar no site e confirmar score real
        print(f"🔁 {palavra} → confirmado no site com score {score_checado}")

def adicionar_chute_site(palavra):
    print(f"\n📤 Enviando chute direto: '{palavra}'")
    chute, score = enviar_chute_site(palavra)

    if score is None:
        print(f"⏭️ Ignorado: '{chute}' não retornou score (talvez repetido ou erro).")
        return

    memory[chute] = score
    get_embedding(chute)
    print(f"📥 Registrado: '{chute}' com score {score}")


In [22]:
time.sleep(5)
def loop_automatico():
    reenviar_chutes_para_interface()
    while True:
        proximas = sugerir_proximo()
        print(proximas)
        for i in range(len(proximas)):
            print(f"\nEnviando chute: {proximas[i]}")
            chute, score = enviar_chute_site(proximas[i])
            print(f"🔍 chute: {chute} (score={score})")
        
            if score is not None:
                adicionar_chute(chute, score)
            else:
                print("⏭️ Ignorando chute inválido ou repetido.")
            try:
                if score <= 10:
                    print("🎉 Palavra secreta encontrada:", chute)
                    break
            except:
                print("⚠️ Erro ao verificar score. Continuando...")
        try:
            if score <= 10:
                print("🎉 Palavra secreta encontrada! Encerrando o jogo.")
                break
        except:
            print("⚠️ Erro ao verificar score final. Continuando...")

In [23]:
listaPalavras = []

if len(listaPalavras) != 0:
    print("⚠️ Nenhuma palavra foi passada para o loop automático.") 
    for palavra in listaPalavras:
        adicionar_chute_site(palavra)
        print(f"🔄 Enviando chute manual: {palavra}")
else:
    print("🔄 Iniciando jogo")
loop_automatico()


🔄 Iniciando jogo

📤 Reenviando chutes manuais para a interface do jogo...

🎯 Gerando novas sugestões...
🧭 Começando exploração: animal
['animal']

Enviando chute: animal

aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa'animal'
🔍 chute: animal (score=3999)
Chute registrado: 'animal' com score 3999

🎯 Gerando novas sugestões...
🔍 Todos os chutes estão ruins. Explorando novo tema: profession
['profession']

Enviando chute: profession

aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa'profession'
🔍 chute: profession (score=28346)
Chute registrado: 'profession' com score 28346

🎯 Gerando novas sugestões...
🔍 Todos os chutes estão ruins. Explorando novo tema: person
['person']

Enviando chute: person

aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa'person'
🔍 chute: person (score=7476)
Chute registrado: 'person' com score 7476

🎯 Gerando novas sugestões...
🔍 Todos os chutes estão ruins. Explorando novo tema: music
['music']

Enviando chute: music

aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa'mus